In [1]:
import numpy as np
import scipy as sp
import pymongo
import pandas as pd
import nsepy
from datetime import datetime
from datetime import timedelta
from pymongo import MongoClient
from nsepy import get_history

In [2]:
client=MongoClient("mongodb://localhost:27018/",username="stockadm",password="1qaz@WSX")
db=client["stockWarehouse"]
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]
equityListPd = pd.DataFrame(equityList.find({}))
equityListPd.head(5)

,Face Value,Group,ISIN No,Industry,Issuer Name,Security Code,Security Id,Security Name,Secutiry Name,State,Status,_id,optionsTrade
0,INE117A01022,2,Heavy Electrical Equipment,Equity,ABB,500002,ABB India Limited,Active,NaN,A,B,5e6dffa36cef0bbb3f7833ea,NaN
1,INE424A01014,10,,Equity,TPAEC,500004,TORRENT POWER AEC LTD.,Delisted,NaN,NaN,B,5e6dffa36cef0bbb3f7833eb,NaN
2,INE885A01032,1,Auto Parts & Equipment,Equity,AMARAJABAT,500008,AMARA RAJA BATTERIES LTD.,Active,NaN,A,A,5e6dffa36cef0bbb3f7833ec,A
3,INE984C01013,10,Iron & Steel Products,Equity,AKARLAMIN,500005,AKAR LAMINATORS LTD.,Delisted,NaN,NaN,XD,5e6dffa36cef0bbb3f7833ed,NaN
4,INE001A01036,2,Housing Finance,Equity,HDFC,500010,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Active,NaN,A,A,5e6dffa36cef0bbb3f7833ee,A


In [3]:
equityListPd = pd.DataFrame(equityList.find({'Security Name':'Active','State':"A"}))
collectionTime=2000
basicSample=30
cnt=0
my_time = datetime.now()

for rows in equityListPd['Issuer Name'].iteritems():
    print(cnt,rows[1],end="-")
    cnt= cnt + 1
    if stockWarehouse.find({'Symbol':rows[1]}).count() > 1 :
        for j in stockWarehouse.find({'Symbol':rows[1]},{'_id':0,'Date':1}).sort('Date',-1).limit(1) :
            collectionTime=((my_time - j['Date']).days - 1)
            fromDate=j['Date']
    iters=int(collectionTime/basicSample)
    if (collectionTime != 0):
        if collectionTime < 2000:
            basicSample=collectionTime
            iters=1
        dt=datetime.now()
        print("Iterations:",iters)
        for i in range(0,iters):
            
            startDate=dt - timedelta(days=basicSample)
            if( startDate <= fromDate):
                startDate = (fromDate + timedelta(days=1))
            endDate=dt 
            print(i,"Collecting into between",startDate.date(),"-", endDate.date(),"@",datetime.now())
            df=pd.DataFrame(get_history(symbol=rows[1] , start= startDate,end=endDate))
            df.reset_index(inplace=True)
            df['Date']=pd.to_datetime(df['Date'])
            dfLength=(len(df.index))
            if  dfLength > 0 :
                data_dict=df.to_dict("records")
                stockWarehouse.insert_many(data_dict)
                tmpdf=pd.DataFrame(stockWarehouse.find({"Symbol":rows[i],"Date":{"$lte":endDate,"$gte":startDate}}))
                print(df.index.size,tmpdf.index.size)
              
            else :
                print("Marking as Inactive")
                equityList.update_many({'Issuer Name':rows[1]},{'$set': {'Security Name':'Inactive'}})
                break;
            dt=(startDate - timedelta(days=1))


0 ABB-

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # Remove the CWD from sys.path while we load stuff.


Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:34:50.416572
6 0
1 AMARAJABAT-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:34:53.157165
6 0
2 HDFC-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:34:53.514413
6 0
3 ASIANHOTNR-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:34:54.623138
6 0
4 ATUL-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:34:55.403871
6 0
5 AUTOLITIND-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:34:55.950680
6 0
6 BAJAJHIND-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:34:56.487007
6 0
7 BAJFINANCE-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:34:57.339699
6 0
8 BALRAMCHIN-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:34:58.019832
6 0
9 BANCOINDIA-Iterations: 1
0 Collect

6 0
77 LICHSGFIN-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:35:26.693666
6 0
78 LUPIN-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:35:27.183576
6 0
79 LYKALABS-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:35:27.499821
6 0
80 RAMCOCEM-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:35:28.154143
6 0
81 MAHSEAMLES-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:35:28.647139
6 0
82 MAHSCOOTER-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:35:29.048691
6 0
83 MANALIPETC-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:35:29.382943
6 0
84 MIRCELECTR-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:35:29.845497
1 0
85 CENTENKA-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:35:30.264051
6 0
86 MOREPE

6 0
152 GNFC-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:36:12.861653
6 0
153 SANOFI-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:36:13.248986
6 0
154 PFIZER-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:36:14.003110
6 0
155 GSFC-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:36:14.399908
6 0
156 HINDUNILVR-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:36:14.725833
6 0
157 AKZOINDIA-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:36:15.057040
6 0
158 NOCIL-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:36:15.600502
6 0
159 TATACHEM-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:36:15.931918
6 0
160 TNPETRO-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:36:16.389589
6 0
161 ZUARIGLOB-Itera

...

6 0
452 SOMANYCERA-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:38:33.009015
6 0
453 CGCL-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:38:33.267610
6 0
454 FDC-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:38:33.751075
6 0
455 MARICO-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:38:34.041849
6 0
456 VIDHIING-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:38:34.331647
6 0
457 PRAENG-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:38:34.756306
6 0
458 POLYMED-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:38:35.023352
6 0
459 ATULAUTO-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:38:35.572550
6 0
460 AMBIKCO-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:38:35.945419
6 0
461 SWELECTES-Itera

6 0
527 ASTRAMICRO-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:08.074680
6 0
528 RADICO-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:08.362607
6 0
529 SHRIRAMCIT-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:08.633255
6 0
530 MARUTI-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:08.876316
6 0
531 UCOBANK-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:09.121321
6 0
532 BAGFILMS-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:09.482276
1 0
533 JSL-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:09.963909
6 0
534 SUPRAJIT-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:10.262891
6 0
535 IGL-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:10.538080
6 0
536 TVTODAY-Itera

6 0
603 HOVS-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:36.651944
6 0
604 ACE-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:36.972440
6 0
605 GEECEE-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:37.315671
6 0
606 GMRINFRA-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:38.248931
6 0
607 MAHINDCIE-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:38.573931
6 0
608 TECHM-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:39.041107
6 0
609 FIEMIND-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:39.428860
6 0
610 DCBBANK-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:40.019008
6 0
611 SHIVAMAUTO-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:39:40.569178
1 0
612 NAUKRI-Iteratio

6 0
679 VGUARD-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:14.218170
6 0
680 RECLTD-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:14.523031
6 0
681 GAMMNINFRA-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:14.807128
6 0
682 TWL-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:15.084375
6 0
683 KIRIINDUS-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:15.421958
6 0
684 BAJAJFINSV-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:15.908065
6 0
685 BAJAJ-AUTO-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:16.169133
6 0
686 RPGLIFE-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:16.523331
6 0
687 RBL-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:16.812528
6 0
688 RANEENGINE

6 0
755 VASWANI-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:53.801933
1 0
756 PGEL-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:54.212122
6 0
757 UJAAS-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:55.273743
6 0
758 ORIENTREF-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:55.677467
6 0
759 MCX-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:56.158847
6 0
760 SCHNEIDER-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:56.714396
6 0
761 NBCC-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:57.125063
6 0
762 MTEDUCARE-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:57.649276
6 0
763 TBZ-Iterations: 1
0 Collecting into between 2020-06-17 - 2020-06-25 @ 2020-06-25 16:40:58.402087
6 0
764 VSSL-Iterations: 1
0 

In [4]:
reference=int(stockWarehouse.find({'Symbol':'SBIN'}).count()) - 100
print(reference)
tmp_count=0
for i in equityList.find({},{'Issuer Name':1}):
    #print(i['Issuer Name'])
    tmp_count=(int(stockWarehouse.find({'Symbol':i['Issuer Name']}).count()))
    if tmp_count >= reference:
        print(i['Issuer Name'])
        equityList.update_many({'Issuer Name':i['Issuer Name']},{'$set': {'State':"A"}})
    

1330
ABB
AMARAJABAT
HDFC


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


ANSALAPI
ASIANHOTNR
ATUL
AUTOLITIND
BAJAJHIND
BAJFINANCE
BALRAMCHIN
BANCOINDIA
CENTURYTEX
BASF
BATAINDIA
BANARISUG
BEL
BEML
BEPL
BLUESTARCO
BPL
CENTEXT
CESC
CHAMBLFERT
EXIDEIND
CIPLA
DICIND
CRISIL
DABUR
DALMIASUG
ARVIND
BALLARPUR
BHEL
HINDPETRO
IFCI
MTNL
MRPL
CHENNPETRO
RELCAPITAL
SAIL
TITAN
IDBI
DCW
DHAMPURSUG
DRREDDY
EIDPARRY
ELECTCAST
SBIN
ESABINDIA
UFLEX
FOSECOIND
GODREJIND
KANSAINER
GHCL
HCL-INSYS
HDFCBANK
HEROMOTOCO
HFCL
HCC
HINDOILEXP
HSIL
HINDZINC
INDIAGLYCO
INDORAMA
INFY
INGERRAND
ATFL
JISLJALEQS
JINDALPOLY
JSWSTEEL
KAJARIACER
KAKATCEM
KSL
WHIRLPOOL
KIRLOSBROS
KIRLOSIND
KOTAKBANK
LGBBROSLTD
TRENT
LAXMIMACH
LICHSGFIN
LUPIN
LYKALABS
RAMCOCEM
MAHSEAMLES
MAHSCOOTER
MANALIPETC
MIRCELECTR
CENTENKA
MOREPENLAB
MRF
HEIDELBERG
NCC
NAHARSPING
GRASIM
PEL
NIITLTD
ONGC
OILCOUNTUB
ORIENTHOT
ORIENTBANK
RELIANCE
BIRLACORPN
PIDILITIND
PRIMESECU
RAYMOND
SURYAROSNI
RSWM
RAMANEWS
RALLIS
JINDALSAW
JKLAKSHMI
SHREECEM
RELINFRA
TATAPOWER
SPMLINFRA
SUNDRMFAST
SUNFLAG
SUPPETRO
SWARAJENG
TATAELXSI
ACC
TH